In [38]:
from boto.s3.connection import S3Connection
from boto.s3.key import Key
import StringIO


conn = S3Connection(host="s3-us-west-1.amazonaws.com")
bucket = conn.get_bucket('w205-rcordell-assignment4')
key = bucket.get_key('WC2015.csv')
contents = StringIO.StringIO(key.get_contents_as_string())

In [39]:
from whoosh.index import create_in
from whoosh.fields import *
schema = Schema(content=TEXT(stored=True), 
				tweetid=ID(stored=True), 
				user=TEXT(stored=True), 
				userid=ID(stored=True), 
				date=DATETIME(stored=True))


In [40]:
ix = create_in("indexdir", schema)
writer = ix.writer()

In [41]:
from time import mktime
from time import gmtime
from datetime import datetime
first = True
while True:
    line = contents.readline()
    if not line:
        break
    if first:
        first = False
        continue
    fields = line.split("|")
    writer.add_document(content=unicode(fields[4]),
                        tweetid=unicode(fields[0]),
                        userid=unicode(fields[1]), 
                        user=unicode(fields[3]), 
                        date=datetime.fromtimestamp(mktime(gmtime(int(fields[5])))))



In [42]:
writer.commit()
contents.close()
key.close()

In [57]:
from whoosh.qparser import QueryParser
with ix.searcher() as searcher:
    query = QueryParser("content", ix.schema).parse(u"- RT")
    results = searcher.search(query)
    print results[0]

<Hit {'content': u'Keep the flag flying RT- RT #ENG #FIFAWWC FRANTASTIC pic.twitter.com/59tA3KsZWI', 'date': datetime.datetime(2015, 7, 4, 23, 43, 28), 'userid': u'146234261', 'user': u'Pinkkimmy1', 'tweetid': u'617463766771474432'}>
